[TORCHTEXT 라이브러리로 텍스트 분류하기](https://tutorials.pytorch.kr/beginner/text_sentiment_ngrams_tutorial.html)

In [ ]:
"""
텍스트 분류

반복자(iterator)로 가공되지 않은 데이터에 접근
가공되지 않은 텍스트 문장들을 모델 학습에 사용할 수 있게 torch.Tensor로 변환하는 데이터 처리 파이프라인 생성
torch.utils.data.DataLoader를 사용하여 데이터를 섞고 반복하기
"""

## 기초 데이터셋 반복자에 접근하기

In [2]:
!pip install portalocker

In [4]:
"""
torchtext 라이브러리: 가공되지 않은 텍스트 문장들을 만드는 데이터셋 반복자 제공
    예) 레이블과 문장의 튜플 형태로 가공되지 않은 데이터 생성
"""

import torch
from torchtext.datasets import AG_NEWS
train_iter = iter(AG_NEWS(split='train'))

In [5]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [7]:
next(train_iter)

(3,
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.')

In [8]:
next(train_iter)

(3,
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.")

## 데이터 처리 파이프라인 준비

In [9]:
"""
torchtext 라이브러리의 가장 기본적인 구성요소

토크나이저 및 어휘집을 사용한 일반적인 NLP 데이터 처리의 예시 보기
    1. 가공되지 않은 학습 데이터셋으로 어휘집 생성
        ㄴ 토큰의 목록 또는 반복자를 받는 내장 팩토리 함수 build_vocab_from_iterator 사용
        ㄴ 사용자는 어휘집에 추가할 특수 기호를 전달할 수도 있다
"""

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

# iterator를 이용하여 vocab 만들기
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

# OOV token에 대해서 인덱스 부여
vocab.set_default_index(vocab["<unk>"])

In [10]:
# 어휘집 블록은 토큰 목록을 정수로 변환한다
vocab(['here', 'is', 'an', 'example'])

[475, 21, 30, 5297]

In [11]:
# 토크나이저, 어휘집을 갖춘 텍스트 처리 파이프라인 준비
# 텍스트/레이블 파이프라인은 데이터셋 반복자로부터 얻어온 raw한 문장 데이터를 처리하기 위해 사용
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [12]:
"""
텍스트 파이프라인
    vocab에 정의된 룩업 테이블에 기반해 텍스트 문장을 정수 목록으로 변환
    레이블 파이프라인은 레이블을 정수로 변환
"""

# 예)
print(text_pipeline('here is the an example'))
print(label_pipeline('10'))

[475, 21, 2, 30, 5297]
9


## 데이터 배치 및 반복자 생성

In [14]:
"""
getitem() 및 len() 프로토콜을 구현한 맵 형태의 데이터셋으로 동작

모델로 보내기 전, collate_fn 함수는 DataLoader로부터 생성된 샘플 배치로 동작
collate_fn
    입력: DataLoader에 배치 크기가 있는 배치 데이터
    입력을 미리 선언된 데이터 처리 파이프라인에 따라 처리

예)
1. 주어진 데이터 배치의 텍스트 항목들은 리스트에 담긴다
2. 리스트에 담긴 텍스트 항목들은 nn.Embedding의 입력을 위한 하나의 tensor로 합쳐진다

* offset은 텍스트 tensor에서 개별 시퀀스 시작 인덱스를 표현하기 위한 구분자이다.
* 레이블은 개별 텍스트 항목의 레이블을 저장하는 tensor이다.
"""

from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    # offset: 텍스트 tensor에서 개별 시퀀스 시작 인덱스를 표현하기 위한 구분자
    label_list, text_list, offsets = [], [], [0]

    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))

    label_list = torch.tensor(label_list, dtype=torch.int64)

    # cumsum: cumulative sum
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)

    # nn.Embedding의 입력을 위한 하나의 Tensor로 합치는 과정
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(
    train_iter,
    batch_size=8,
    shuffle=False,
    collate_fn = collate_batch
)

## 모델 정의

모델은 nn.EmbeddingBag 레이어 및 분류를 위한 선형 레이어로 구성된다

`nn.EmbeddingBag`은 기본으로 가방의 평균 값을 계산한다

텍스트의 길이를 offset으로 저장하고 있으므로, 패딩이 필요하지 않다

![](https://tutorials.pytorch.kr/_images/text_sentiment_ngrams_model.png)

In [15]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        embed_dim,
        num_class,
    ):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        # text: size of the dictionary of embeddings
        # offsets: size of each embedding vector
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

## 인스턴스 생성

4종류의 레이블, 클래스의 개수도 4개이다

```
1 : World (세계)
2 : Sports (스포츠)
3 : Business (경제)
4 : Sci/Tec (과학/기술)
```

In [16]:
# 임베딩 차원이 64인 모델 만들기
# 어휘집 크기 = 어휘집 길이, 클래스 개수 = 레이블 개수

train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(
    vocab_size,
    emsize,
    num_class
).to(device)

## 모델을 학습하고 결과 평가하는 함수 정의

In [21]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500

    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)

        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}|{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader), total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

## 데이터셋을 분할하고 모델 수행

In [22]:
"""
원본 AG_NEWS에는 val 데이터가 포함되어 있지 않다
    ㄴ 학습 95: 검증 5로 분리하기
    ㄴ torch.utils.data.dataset.random_split 함수 사용하여 분리

CrossEntropyLoss: nn.LogSoftmax() + nn.NLLLoss()
"""

from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

EPOCHS = 10
LR = 5
BATCH_SIZE = 64

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

total_accu = None
train_iter, test_iter = AG_NEWS()

# convert iterable-style dataset to map-style dataset
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    # [num_train, num_val] 로 개수 설정
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch
)

valid_dataloader = DataLoader(
    split_valid_,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch
)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)

    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 60)
    print('| end of epoch {:3d} | time: {:5.2f}s |'
          'valid accuracy {:8.3f}'.format(epoch,
                                          time.time() - epoch_start_time,
                                          accu_val)
    )
    print('-' * 60)

| epoch   1 |   500| 1782 batches | accuracy    0.678
| epoch   1 |  1000| 1782 batches | accuracy    0.856
| epoch   1 |  1500| 1782 batches | accuracy    0.876
------------------------------------------------------------
| end of epoch   1 | time: 10.41s |valid accuracy    0.885
------------------------------------------------------------
| epoch   2 |   500| 1782 batches | accuracy    0.898
| epoch   2 |  1000| 1782 batches | accuracy    0.901
| epoch   2 |  1500| 1782 batches | accuracy    0.900
------------------------------------------------------------
| end of epoch   2 | time: 10.39s |valid accuracy    0.900
------------------------------------------------------------
| epoch   3 |   500| 1782 batches | accuracy    0.916
| epoch   3 |  1000| 1782 batches | accuracy    0.914
| epoch   3 |  1500| 1782 batches | accuracy    0.913
------------------------------------------------------------
| end of epoch   3 | time: 11.35s |valid accuracy    0.906
--------------------------------

## 임의의 뉴스로 평가하기

In [23]:
# 다른 뉴스로 테스트하기(골프 뉴스)
ag_news_label = {
    1: "World",
    2: "Sports",
    3: "Business",
    4: "Sci/Tec"
}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. - Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news
